In [2]:
import pandas as pd
import numpy as np
import itertools
from scipy import stats
import string
from tqdm import tqdm
import os

# 读取固有

In [3]:
# 读取树
GO_terms = pd.read_csv('GSVA_Term_ID对应表.csv',header=0,index_col=0)
# GO_terms

# 读取记录目标
target_tree = pd.read_csv('GO目标分级.csv',header=0,index_col=0)
# target_tree

# 设置循环素材

In [4]:
def main(p_ret_file_name,cor_ret_file_name,p_shreshold,open_file_name):
    # p_ret_file_name = '../2.预处理：GSVA与RF相关二维表/JSPH/→JSPH_GSVA_二维_可信度表.csv'
    # cor_ret_file_name = '../2.预处理：GSVA与RF相关二维表/JSPH/→JSPH_GSVA_二维_相关系数表.csv'
    # p_shreshold = 0.05
    # open_file_name = 'demo'
    
    # 读取前一步的结果
    p_ret = pd.read_csv(p_ret_file_name,header=0,index_col=0)
    cor_ret = pd.read_csv(cor_ret_file_name,header=0,index_col=0)

    # 检查前一步中的GSVA条目在不在 GO拓扑结构树中
    nameList = GO_terms.index.tolist()
    orphan = []
    for i in p_ret.index.tolist():
        if i not in nameList:
            orphan.append(i)

    print(f'打开的p_ret和cor_ret有{len(orphan)}个在GO拓扑树中没找到')
    
    # 在 预结果表 中添加p_ret的列索引
    feature = p_ret.columns.tolist()
    for i in feature:
        target_tree[i] = 0

    
    # 预创建结果表
    ret1 = target_tree.copy(deep=True)
    ret2 = target_tree.copy(deep=True)
    ret3 = target_tree.copy(deep=True)
    
    # 开始正经处理
    # 循环判断每一个特征，得到  计数表
    for iteration in tqdm(range(len(feature))):
        f = feature[iteration]

        for i in p_ret.index:
            # 一旦满足条件，p<0.05，寻找该term在大表中的位置，并判断其是否在目标层级中，然后，把其添加到结果列表里
            if p_ret[f][i] <= p_shreshold:
                json = GO_terms['class2level'][i]

                if json!='not_find':
                    json = eval(json)
                    # 接下来就是判断目标计数表中的每一项的ID，在不在JSON中，在就+1
                    for item in target_tree.index:
                        if item in json:

                            # ---------------------------------------------
                            # 得到  计数表
                            ret1.loc[item,f] = ret1[f][item]+1

                            # ---------------------------------------------
                            # 得到  平均相关程度表
                            if ret2.loc[item,f]==0:
                                ret2.loc[item,f] = cor_ret[f][i]
                            # 如果已经有值了，比如原来相关度是0.3，新相关度是-0.2，那么最后的相关度是 (0.3+|-0.2|)/2
                            else:
                                ret2.loc[item,f] = (ret2[f][item]+ abs(cor_ret[f][i]))/2

                            # ---------------------------------------------
                            # 得到  平均p值表
                            if ret3.loc[item,f]==0:
                                ret3.loc[item,f] = p_ret[f][i]
                            else:
                                ret3.loc[item,f] = (ret3[f][item]+ abs(p_ret[f][i]))/2
                                
                                
    # 获取想要保存的目录                            
    roooooot_len = len(p_ret_file_name) - len(p_ret_file_name.split("/")[-1])
    roooooot = p_ret_file_name[:roooooot_len]
    
    # 将所有结果保存
    save_name1 = '→'+open_file_name+'_p'+str(p_shreshold)+'_结果① 分类计数表.csv'
    save_name2 = '→'+open_file_name+'_p'+str(p_shreshold)+'_结果② 分类平均相关度表.csv'
    save_name3 = '→'+open_file_name+'_p'+str(p_shreshold)+'_结果③ 分类平均p值表.csv'

    ret1.to_csv(os.path.join(roooooot,save_name1), index=True)
    ret2.to_csv(os.path.join(roooooot,save_name2), index=True)
    ret3.to_csv(os.path.join(roooooot,save_name3), index=True)

In [5]:
main(r'../→GSVA后数据/JSPH/GSVA_JSPH_二维_可信度表.csv',
     r'../→GSVA后数据/JSPH/GSVA_JSPH_二维_相关系数表.csv',
     0.05,
     'JSPH'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.54it/s]


In [6]:
main(r'../→GSVA后数据/RBRT/GSVA_RBRT_二维_可信度表.csv',
     r'../→GSVA后数据/RBRT/GSVA_RBRT_二维_相关系数表.csv',
     0.05,
     'RBRT'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:21<00:00,  1.05s/it]


In [7]:
main(r'../→GSVA后数据/RBRT_GBM/GSVA_RBRT_GBM_二维_可信度表.csv',
     r'../→GSVA后数据/RBRT_GBM/GSVA_RBRT_GBM_二维_相关系数表.csv',
     0.05,
     'RBRT_GBM'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.75it/s]


In [8]:
main(r'../→GSVA后数据/RBRT_LGG/GSVA_RBRT_LGG_二维_可信度表.csv',
     r'../→GSVA后数据/RBRT_LGG/GSVA_RBRT_LGG_二维_相关系数表.csv',
     0.05,
     'RBRT_LGG'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.18it/s]


In [9]:
main(r'../→GSVA后数据/TCGA/GSVA_TCGA_二维_可信度表.csv',
     r'../→GSVA后数据/TCGA/GSVA_TCGA_二维_相关系数表.csv',
     0.05,
     'TCGA'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.00s/it]


In [10]:
main(r'../→GSVA后数据/TCGA_GBM/GSVA_TCGA_GBM_二维_可信度表.csv',
     r'../→GSVA后数据/TCGA_GBM/GSVA_TCGA_GBM_二维_相关系数表.csv',
     0.05,
     'TCGA_GBM'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.08it/s]


In [11]:
main(r'../→GSVA后数据/TCGA_LGG/GSVA_TCGA_LGG_二维_可信度表.csv',
     r'../→GSVA后数据/TCGA_LGG/GSVA_TCGA_LGG_二维_相关系数表.csv',
     0.05,
     'TCGA_LGG'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.11it/s]


In [12]:
main(r'../→GSVA后数据/ALL/GSVA_ALL_二维_可信度表.csv',
     r'../→GSVA后数据/ALL/GSVA_ALL_二维_相关系数表.csv',
     0.05,
     'ALL'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:21<00:00,  1.08s/it]


In [13]:
main(r'../→GSVA后数据/ALL_LGG/GSVA_ALL_LGG_二维_可信度表.csv',
     r'../→GSVA后数据/ALL_LGG/GSVA_ALL_LGG_二维_相关系数表.csv',
     0.05,
     'ALL_LGG'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:14<00:00,  1.34it/s]


In [14]:
main(r'../→GSVA后数据/ALL_GBM/GSVA_ALL_GBM_二维_可信度表.csv',
     r'../→GSVA后数据/ALL_GBM/GSVA_ALL_GBM_二维_相关系数表.csv',
     0.05,
     'ALL_GBM'
    )

打开的p_ret和cor_ret有0个在GO拓扑树中没找到


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 18.28it/s]
